# OpenAI Agents SDK with MCP Server

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/03/notebooks/open-meteo-mcp.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/03/notebooks/open-meteo-mcp.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

Note: To explore the Open Meteo server from the MCP model inspector, run `npx @modelcontextprotocol/inspector`. Within the inspector, set the command to `npx` and parameters to `-y open-meteo-mcp-server`. Note: You'll need NodeJS installed to run the npx command. (v22.19.0 recommended)

## Install pre-requisites

In [ ]:
!uv pip install openai-agents==0.4.2

Resolved 197 packages in 3ms
Installed 10 packages in 34ms                               
 + colorama==0.4.6
 + cryptography==46.0.3
 + griffe==1.15.0
 + httpx-sse==0.4.3
 + mcp==1.25.0
 + openai-agents==0.4.2
 + pydantic-settings==2.12.0
 + pyjwt==2.10.1
 + sse-starlette==3.1.2
 + types-requests==2.32.4.20260107


## Set the OpenAI API Key environment variable

In [ ]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  load_dotenv()

## MCP Server Setup

In [2]:
import logging

# Suppress MCP client warnings about non-JSON messages from the server
logging.getLogger("mcp").setLevel(logging.CRITICAL)

from agents import Agent, Runner
from agents.mcp import MCPServerStdio

### List Available Tools

In [3]:
async with MCPServerStdio(
    name="Open-Meteo Weather Server",
    params={
        "command": "npx",
        "args": ["-y", "open-meteo-mcp-server"],
    },
) as server:
    tools = await server.list_tools()
    print(f"Available tools: {[tool.name for tool in tools]}")

Available tools: ['weather_forecast', 'weather_archive', 'air_quality', 'marine_weather', 'elevation', 'flood_forecast', 'seasonal_forecast', 'climate_projection', 'ensemble_forecast', 'geocoding', 'dwd_icon_forecast', 'gfs_forecast', 'meteofrance_forecast', 'ecmwf_forecast', 'jma_forecast', 'metno_forecast', 'gem_forecast']


### Simple Weather Query

In [5]:
async with MCPServerStdio(
    name="Open-Meteo Weather Server",
    params={
        "command": "npx",
        "args": ["-y", "open-meteo-mcp-server"],
    },
) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )
    
    result = await Runner.run(agent, "What's the weather forecast for Minnesota this week?")
    print(result.final_output)

Minnesota is a big state and conditions vary, so here’s a **representative statewide forecast using the Twin Cities (Minneapolis–St. Paul) area** for the next 7 days (America/Chicago):

**This week (daily highs/lows, precip, max wind)**
- **Fri 1/23:** Mostly cloudy, **-7°F / -20°F**, **0.00 in**, wind up to **11 mph**
- **Sat 1/24:** Mostly cloudy, **1°F / -15°F**, **0.00 in**, wind up to **6 mph**
- **Sun 1/25:** Mostly cloudy, **6°F / -4°F**, **0.00 in**, wind up to **9 mph**
- **Mon 1/26:** Mostly cloudy, **12°F / -6°F**, **0.00 in**, wind up to **9 mph**
- **Tue 1/27:** Mostly cloudy, **13°F / 4°F**, **0.00 in**, wind up to **13 mph**
- **Wed 1/28:** Mostly cloudy, **9°F / 1°F**, **0.00 in**, wind up to **9 mph**
- **Thu 1/29:** Mostly cloudy, **11°F / 3°F**, **0.00 in**, wind up to **4 mph**

**Overall:** A cold, mostly cloudy week with **no measurable precipitation showing** in this forecast and **light-to-moderate winds**.

If you tell me your city/ZIP (e.g., Duluth, Rochester,

### Air Quality Query

In [9]:
async with MCPServerStdio(
    name="Open-Meteo Weather Server",
    params={
        "command": "npx",
        "args": ["-y", "open-meteo-mcp-server"],
    },
) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )
    
    result = await Runner.run(agent, "What's the current air quality in Los Angeles?")
    print(result.final_output)

Los Angeles air quality (latest hourly value, **2026-01-15 00:00** local time, America/Los_Angeles):

- **PM2.5:** 22.6 µg/m³  
- **PM10:** 22.7 µg/m³  
- **Ozone (O₃):** 1 µg/m³  
- **Nitrogen dioxide (NO₂):** 64.1 µg/m³  
- **Carbon monoxide (CO):** 307 µg/m³  
- **Sulphur dioxide (SO₂):** 9.7 µg/m³  

If you want, tell me your neighborhood/ZIP in LA and I can check a closer point estimate (levels can vary a lot across the basin).


### Historical Weather Data

In [8]:
async with MCPServerStdio(
    name="Open-Meteo Weather Server",
    params={
        "command": "npx",
        "args": ["-y", "open-meteo-mcp-server"],
    },
) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )
    
    result = await Runner.run(agent, "What was the average temperature in Tokyo during January 2024?")
    print(result.final_output)

Using ERA5 reanalysis data for central Tokyo (Asia/Tokyo), the **average temperature in Tokyo during January 2024** was **about 5.8 °C**.

(Computed as the mean of each day’s average temperature: \((T_{\max}+T_{\min})/2\) averaged over **2024-01-01 to 2024-01-31**.)
